<center>

###**Implementación Attention is All You Need.ipynb** <br><br>

**Froilan Moya Robles**

**Brian Morera Madriz**

**Allan Murillo Gonzalez**

**Esteban Esquivel Barboza**

**Michael Pérez Ocampo**<br><br>

**05 de Nomviembre del 2023**<br><br>

**Instituto Tecnológico de Costa Rica**

**Maestría en Ciencias de la Computación**

</center>